# Restructuring the redox data

We basically have 3 types of data:
    
1. The well map that describes which well contains which experiments
2. The Metadata of the redox measurements (Date, intended temperature, gas control, ....)
3. The redox mesurements themselves

We need to keep these data in separate files.

I have already converted the files to CSV files for you (again, with tab as column delimiter).

We have the following files:

In [1]:
from os import listdir
sorted( listdir( 'measurements/' ) )

['01_INIMS_ExiCyToxAssay_RTG_Lars_Binkle_C01_C02__20170620_101455.csv',
 '02_INIMS_ExiCyToxAssay_RTG_Lars_Binkle_C01_C02__20170621_102938.csv',
 '03_INIMS_ExiCyToxAssay_RTG_Lars_Binkle_C01_C02__20170622_103210.csv',
 '04_INIMS_ExiCyToxAssay_RTG_Lars_Binkle_C01_C02_20170623_103550.csv',
 '05_INIMS_ExiCyToxAssay_RTG_Lars_Binkle_C01_C02_20170627_105726.csv',
 '06_INIMS_ExiCyToxAssay_RTG_Lars_Binkle_C01_C02_20170629_095649.csv',
 '07_INIMS_ExiCyToxAssay_RTG_Lars_Binkle_C01_C02_20170630_102212.csv',
 'Plate_Map_INIMS_ExiCyToxAssay_RTG_Lars_Binkle_C01_C02_joern.csv',
 'masterkurs2018_n2a_differentiated_20180108_20180109_day1.csv',
 'masterkurs2018_n2a_differentiated_20180108_20180109_day2.csv']

The plate map is in the file Plate_Map_INIMS_ExiCyToxAssay_RTG_Lars_Binkle_C01_C02_joern.csv.
After talking to Lars I now now how to decode this matric.

Planned structure:

<table>
<thead>
<tr><th>well</th><th>label</th></tr>
</thead>
<tbody>
<tr><td>A2</td><td>Supernatant</td></tr>
<tr><td>F3</td><td>APV [ 50 μM + Glu [ 50 μM ]</td></tr>
</tbody>
</table>

And each well should be its own line because each well is a single observation.

In [2]:
#helper function
def determine_wells( wells_definition ):
    #we assume that wells are numbered from A-H and from 1 to 12
    #furthermore we assume that the well definition is a string
    #of the form A1:H12 where the label before the colon is the
    #top-left corner and the label after the colon is the bottom-left
    #corner of the area.
    corners = wells_definition.split( ':' )
    assert len( corners ) == 2, 'Definition of well range is invalid'
    letter1 = corners[ 0 ][ 0 ] #first corner name and then first letter in the string: A12 -> A
    letter2 = corners[ 1 ][ 0 ]
    number1 = int( corners[ 0 ][ 1: ] ) #first corner name and then all all letters after the first letter: A12 -> 12
    number2 = int( corners[ 1 ][ 1: ] )
    
    result = []
    while letter1 <= letter2:
        while number1 <= number2:
            result.append( letter1 + str( number1 ) )
            number1 += 1
        letter1 = chr( ord( letter1 ) + 1 ) #counting one letter up by using the UTF-8 representation
        number1 = int( corners[ 0 ][ 1: ] ) #restore the old number for the inner loop
    return result

#main tidy function for well labels
def tidy_well_data( filename, outfile_name ):
    labels = {} # well -> label
    with open( filename ) as f:
        for line in f:
            tokens = line.rstrip( '\n' ).split( '\t' ) #always do .rstrip('\n') to get rid of the newline marker
            if tokens[ 0 ] == 'Wells': #check for header line
                continue #ignore header
            wells = determine_wells( tokens[ 0 ] ) #convert ranges of wells into individual well names
            label = tokens[ 1 ].replace( '\\mu', 'μ' ) # I prefer to have the real μ character instead of \mu
            for well in wells:
                labels[ well ] = label
    
    with open( outfile_name, 'w' ) as out:
        out.write( 'well\tlabel\n' ) #header
        for well, label in labels.items():
            out.write( '{}\t{}\n'.format( well, label ) )

In [3]:
tidy_well_data( 'measurements/Plate_Map_INIMS_ExiCyToxAssay_RTG_Lars_Binkle_C01_C02_joern.csv', 'measurements/plate_map.csv' )

Next, let's extract the metadata from the measurements. The output format will be very straightforward:

<table>
<thead>
<tr><th>key</th><th>value</th></tr>
</thead>
<tbody>
<tr><td>Target temperature</td><td>37°C</td></tr>
<tr><td>Date</td><td>2018-01-08</td></tr>
</tbody>
</table>

In [4]:
#helper function
def extract_metadata( filename ):
    result = {}
    
    #these keys have their values in column [4]
    #it is common practise in data cleaning that you explicitly state these details
    #because otherwise it is a lot of work to write code that detects these structures
    #automatically. So it is perfectly fine to use human intelligence here.
    long_distance_key_value_names = [ 'Date', 'Time', 'System', 'User', 'Plate', 'Lid lifter',
                                      'Humidity Casette', 'Name', 'Firmware' ]
    with open( filename ) as f:
        for line in f:
            parts = line.rstrip( '\n' ).split( '\t' )
            if parts[ 0 ] == 'RTG': #this is where the measurements start
                #we need to fast-forward till the entry with the End time
                while not line.startswith( 'End Time' ):
                    line = f.readline() #we now do the line fetching ourselves
                    
                end_time = line.split( '\t' )[ 4 ]
                result[ 'End Time' ] = end_time
                return result # we only care about the metadata

            key, some_value, unit = parts[ 0 ], parts[ 4 ], parts[ 5 ]
            if ':' in key: #check if there is some relationship that is marked with ':'
                key2, value = key.split( ':' )
                if key2 in long_distance_key_value_names: #check if we need to look for the value far to the right
                    result[ key2 ] = some_value + " " + unit
                else:
                    result[ key2 ] = value
            elif some_value != '':
                result[ key ] = some_value + unit
                
    assert False, 'We should never reach this line.' #we should exit after we read the 'End Time' entry

def tidy_metadata( filename, outfile_name ):
    with open( outfile_name, 'w' ) as out:
        out.write( 'key\tvalue\n' )
        for key, value in extract_metadata( filename ).items():
            out.write( '{}\t{}\n'.format( key, value ) )

In [5]:
tidy_metadata( 'measurements/masterkurs2018_n2a_differentiated_20180108_20180109_day1.csv', 'measurements/tidy_metadata_day1.csv' )
tidy_metadata( 'measurements/masterkurs2018_n2a_differentiated_20180108_20180109_day2.csv', 'measurements/tidy_metadata_day2.csv' )

And let us finally proceed to the actual measurements. EI suggest that each file should represent one day. Each row will be one time point. Then we will have the followin structure:

<table>
<thead>
<tr>
 <th>cycle number</th>
 <th>Time [s]</th>
 <th>CO2 %</th>
 <th>O2 %</th>
 <th>Temp [°C]</th>
 <th>A1</th>
 <th>A2</th>
 <th>...</th>
 <th>H12</th>
 </tr>
</thead>
<tbody>
<tr>
 <td>1</td>
 <td>0</td>
 <td>4.8</td>
 <td>19.7</td>
 <td>37.03</td>
 <td>1667</td>
 <td>2480</td>
 <td>2656</td>
 <td>2676</td>
</tr>
<tr>
 <td>2</td>
 <td>1799</td>
 <td>5.0</td>
 <td>19.7</td>
 <td>37.1</td>
 <td>34782</td>
 <td>51977</td>
 <td>50725</td>
 <td>50916</td>

</tr>
</tbody>
</table>

In [6]:
from collections import defaultdict

def tidy_data( filename, output_filename ):
    with open( filename ) as f:
        for line in f:
            if line.startswith( 'RTG' ):
                break #okay, we can now proceed with the actual measurements
                
        cycle_numbers = f.readline().rstrip( '\n' ).split( '\t' )
        assert cycle_numbers[ 0 ] == 'Cycle Nr.', 'First field after RTG is not Cycle Nr.'
        assert int( cycle_numbers[ -1 ] ) == len( cycle_numbers ) - 1, 'Cycle numbers are not 1,2,3,...'
        
        times = f.readline().rstrip( '\n' ).split( '\t' )
        assert times[ 0 ] == 'Time [s]', 'Second field after RTG is not Time [s]'
        
        co2 = f.readline().rstrip( '\n' ).split( '\t' )
        assert co2[ 0 ] == 'CO2 %', 'Third field after RTG is not CO2 %'
        
        o2 = f.readline().rstrip( '\n' ).split( '\t' )
        assert o2[ 0 ] == 'O2 %', 'Fourth field after RTG is not O2 %'
        
        temperature = f.readline().rstrip( '\n' ).split( '\t' )
        assert temperature[ 0 ] == 'Temp. [°C]', 'Fifth field after RTG is not Temperature'
        
        well_measurements = []
        for line in f: #proceed with the remaining lines
            tokens = line.rstrip( '\n' ).split( '\t' )
            if tokens[ 0 ] == '':
                continue #ignore empty lines
            elif tokens[ 0 ] == 'End Time':
                break #time for a output
            
            well_measurements.append( tokens )

        else:
            assert False, 'Expected "End Time" field in file'
        
    #output
    with open( output_filename, 'w' ) as out:
        out.write( 'cycle number\tTime [s]\tCO2 %\tO2 %\tTemp [°C]' )
        for well in well_measurements:
            out.write( "\t" + well[ 0 ] ) #this is the name of the well
        out.write( "\n" )
        for i in range( 1, len( cycle_numbers ) ):
            out.write( '{}\t{}\t{}\t{}\t{}'.format( i, times[ i ], co2[ i ], o2[ i ], temperature[ i ] ) )
            for well in well_measurements:
                out.write( "\t" + well[ i ] )
            out.write( '\n' )

In [7]:
tidy_data( 'measurements/masterkurs2018_n2a_differentiated_20180108_20180109_day1.csv', 'measurements/tidy_data_day1.csv' )
tidy_data( 'measurements/masterkurs2018_n2a_differentiated_20180108_20180109_day2.csv', 'measurements/tidy_data_day2.csv')

And now we can analyse the data!